In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import time

In [2]:
USE_CUDA = torch.cuda.is_available()
torch.cuda.manual_seed(1) if USE_CUDA else torch.manual_seed(1)

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}

In [1]:
import json
import loguru

with open('../configs/cnn_configs.json', 'r') as f:
    configs = json.load(f)
    loguru.logger.info('Loaded the configuration file.')

BATCH_SIZE = configs['BATCH_SIZE']
LR = configs['LR']
MOMENTUM = configs['MOMENTUM']
EPOCHS = configs['EPOCHS']
LOG_INTERVAL = configs['LOG_INTERVAL']

2024-04-05 13:30:12.569 | INFO     | __main__:<module>:6 - Loaded the configuration file.


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        loguru.logger.debug(f'params num: {sum(p.numel() for p in self.parameters() if p.requires_grad)}')

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [21]:
model = Net()
if USE_CUDA:
    model.cuda()

2024-04-04 19:01:16.196 | DEBUG    | __main__:__init__:9 - params num: 21840


In [8]:
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

In [9]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if USE_CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if USE_CUDA:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target).item()
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()
    
    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [11]:
for epoch in range(1, EPOCHS + 1):
    init = time.time()
    train(epoch)
    ending = time.time()
    loguru.logger.debug(f'Time for the epoch: {ending - init:.4f}s.')
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.354873
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.042626
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.233654
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.993994
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.799430
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.756497
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.486018
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.549505
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.604204
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.500301


2024-04-04 18:00:07.568 | DEBUG    | __main__:<module>:5 - Time for the epoch: 17.4860s.



Test set: Average loss: 0.2244, Accuracy: 9348/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.442255
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.464616
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.376070
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.606645
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.455777
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.237125
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.460890
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.289720
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.286341
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.551159


2024-04-04 18:00:21.672 | DEBUG    | __main__:<module>:5 - Time for the epoch: 10.6831s.



Test set: Average loss: 0.1278, Accuracy: 9623/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.392027
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.165389
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.424578
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.509329
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.418495
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.252133
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.298127
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.395802
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.142130
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.293039


...


Test set: Average loss: 0.0574, Accuracy: 9814/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.148728
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.121965
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.217122
Train Epoch: 9 [19200/60000 (32%)]	Loss: 0.172365
Train Epoch: 9 [25600/60000 (43%)]	Loss: 0.114969
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.055023
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.100465
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.119181
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.137933
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.299993


2024-04-04 18:01:59.868 | DEBUG    | __main__:<module>:5 - Time for the epoch: 10.6024s.



Test set: Average loss: 0.0491, Accuracy: 9840/10000 (98%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.208018
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.148983
Train Epoch: 10 [12800/60000 (21%)]	Loss: 0.145555
Train Epoch: 10 [19200/60000 (32%)]	Loss: 0.278376
Train Epoch: 10 [25600/60000 (43%)]	Loss: 0.355840
Train Epoch: 10 [32000/60000 (53%)]	Loss: 0.264135
Train Epoch: 10 [38400/60000 (64%)]	Loss: 0.117086
Train Epoch: 10 [44800/60000 (75%)]	Loss: 0.171919
Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.186838
Train Epoch: 10 [57600/60000 (96%)]	Loss: 0.350923


2024-04-04 18:02:13.953 | DEBUG    | __main__:<module>:5 - Time for the epoch: 10.7101s.



Test set: Average loss: 0.0482, Accuracy: 9846/10000 (98%)



In [16]:
torch.save(model.state_dict(), '../model/cnn/mnist_cnn.pt')

In [19]:
torch.load('../model/cnn/mnist_cnn.pt', map_location=lambda storage, loc: storage)

OrderedDict([('conv1.weight',
              tensor([[[[-2.2801e-01, -2.0178e-01,  1.1392e-01,  1.4996e-01,  6.7549e-02],
                        [-2.4287e-01, -8.0902e-03,  4.7206e-02, -8.0363e-02, -7.8436e-02],
                        [-2.5816e-01, -2.2800e-01, -1.5778e-01, -6.3274e-02, -1.7308e-01],
                        [-1.3900e-01,  1.3632e-02, -5.6343e-02, -1.5601e-01, -2.3128e-01],
                        [-1.1529e-01, -1.5112e-01, -1.7182e-01, -1.0877e-01,  5.7885e-02]]],
              
              
                      [[[ 2.6658e-01,  4.1212e-01,  4.6037e-01,  2.5170e-01,  8.1825e-02],
                        [ 1.3873e-01,  2.6663e-01,  5.7617e-02, -9.3925e-02, -9.6656e-02],
                        [ 2.5818e-01, -1.6053e-01, -3.8164e-01, -1.0151e-01, -2.5368e-01],
                        [-1.1415e-01, -3.3096e-01, -1.5438e-01, -2.7884e-01,  1.0435e-01],
                        [-1.9238e-01, -3.3290e-01, -5.2033e-02,  1.6483e-01,  2.0401e-01]]],
              
           